In [ ]:
import pandas as pd
from acti_motus import Features, Activities
from pathlib import Path


def rotate_by_90_degrees_over_x(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    y_new = -df['acc_z']
    z_new = df['acc_y']

    df['acc_y'] = y_new
    df['acc_z'] = z_new

    return df


files = Path('parquets').glob('*.parquet')

features = Features()
activities = Activities('Sens', orientation=False)

results = []

for file in files:
    gt = file.parent.parent / 'ground_truth' / file.name

    if not gt.exists():
        print(f'Ground truth file does not exist for {file.name}, skipping.')
        continue

    ground_truth = pd.read_parquet(file.parent.parent / 'ground_truth' / file.name)
    df = pd.read_parquet(file)
    df = rotate_by_90_degrees_over_x(df)

    extracted_features = features.extract(df)
    activity, references = activities.detect(extracted_features)

    df = ground_truth.join(activity, how='left')
    results.append(df)

results = pd.concat(results)
results = results.loc[~((results['id'] == 'P17') & (results['ground_truth'] == 'running_fast'))]
results.to_parquet('processed_data.parquet', index=True)

In [ ]:
import sys

sys.path.append('..')

from functions import get_confusion_matrix, get_validity_metrics
import pandas as pd


df = pd.read_parquet('processed_data.parquet')
df = df.loc[df['condition'] == 'laboratory']

annotations = df['ground_truth'].unique().tolist()

rename = {
    'standing': 'stand',
    'sitting': 'sit',
    'lying_f': 'lie',
    'lying_s': 'lie',
    'lying_b': 'lie',
    'walking_slow': 'walk',
    'walking_mod': 'walk',
    'walking_fast': 'walk',
    'running_slow': 'run',
    'running_mod': 'run',
    'running_fast': 'run',
    'cycling_slow': 'bicycle',
    'cycling_mod': 'bicycle',
    'cycling_fast': 'bicycle',
}
df['ground_truth'] = df['ground_truth'].map(rename)
df.loc[df['activity'] == 'move', 'activity'] = 'stand'
df.loc[df['activity'] == 'stairs', 'activity'] = 'walk'

plot = get_confusion_matrix(
    df['ground_truth'], df['activity'], labels=['lie', 'sit', 'stand', 'walk', 'run', 'bicycle']
)  # move, stairs
plot.write_image('laboratory.png', scale=2)

results = get_validity_metrics(df['ground_truth'], df['activity'])
results.to_csv('laboratory.csv')

In [ ]:
import sys

sys.path.append('..')

from functions import get_confusion_matrix, get_validity_metrics
import pandas as pd

df = pd.read_parquet('processed_data.parquet')
df = df.loc[df['condition'] == 'free-living']
df = df.loc[~df['ground_truth'].isin(['start', 'end', 'heel_1', 'heel_2', 'heel_3', 'undefined', 'undefined_covered'])]
annotations = df['ground_truth'].unique().tolist()

rename = {
    'standing_static': 'stand',
    'walking': 'walk',
    'standing_dynamic': 'stand',
    'running': 'run',
    'walking_stairs': 'walk',
    'sitting': 'sit',
    'lying_down': 'lie',
    'cycling_dynamic': 'bicycle',
    'cycling_static': 'bicycle',
    'cycling_standing': 'bicycle',
}
df['ground_truth'] = df['ground_truth'].map(rename)
df.loc[df['activity'] == 'move', 'activity'] = 'stand'
df.loc[df['activity'] == 'stairs', 'activity'] = 'walk'


plot = get_confusion_matrix(
    df['ground_truth'], df['activity'], labels=['lie', 'sit', 'stand', 'walk', 'run', 'bicycle']
)  # move, stairs
plot.write_image('free-living.png', scale=2)

results = get_validity_metrics(df['ground_truth'], df['activity'])
results.to_csv('free-living.csv')